# Simplification

In [1]:
import sympy as sp
# need to import the custom function to print vectors
from vector_expr_printing import vector_latex
# starting point of a vector expression is a vector symbol
from vector_expr import (
    VectorSymbol, VectorExpr, VectorZero, VectorOne, Nabla,
    VecCross, VecDot, Grad, Laplace
)
# also import symbolic vectors
from sympy.vector import CoordSys3D, Vector
# setting the printer to use latex_vector function
from sympy.interactive import printing
printing.init_printing(latex_printer=vector_latex)

In [2]:
a, b, c, d, e, f, g, h = [VectorSymbol(t) for t in 
            ["a", "b", "c", "d", "e", "f", "g", "h"]]
x, y, z = sp.symbols("x:z")

Let's import a few functions from the simplification module:

In [3]:
from vector_simplify import (
    bac_cab, dot_cross, find_bac_cab, find_double_cross, 
    collect, collect_cross_dot, simplify, identities
)

## BAC-CAB rule

We are all familiar with the double cross product rule, also known as **bac-cab** rule:

$$
\vec{a} \times \left(\vec{b} \times \vec{c}\right) = \vec{b} \left(\vec{a} \cdot \vec{c}\right) - \vec{c} \left(\vec{a} \cdot \vec{b}\right)
$$

This module exposes the following function `bac_cab(expr, forward=True, matches=None)`. Specifically:
* `bac_cab(expr, forward=True)` to perform the substitution from the left to the right.
* `bac_cab(expr, forward=False)` to perform the substitution from the right to the elft.

Let's see a few examples:

In [4]:
expr = 2 * x * (a ^ (b ^ c)) + (d ^ (e ^ f))
display(expr, bac_cab(expr))

As we can see, the function went on to substitue all the occurences. Should we wish to target only one occurence, we can pass the target as the second argument:

In [5]:
targets = find_double_cross(expr)
bac_cab(expr, True, targets[0])

Here we used the function `find_double_cross(expr)` to easily find all the occurences, then we selected the one we were interested in substituting.

Another example, to show that the function is able to detect not only `VectorSymbol`s:

In [6]:
expr = a ^ ((2 * (b + c)) ^ (x * d))
display(expr, bac_cab(expr))

Now, let's move backward:

In [7]:
expr = (b * (a & c)) - (c * (a & b)) + 2 * x * y * (d * (e & f)) - (2 * x * y * f * (d & e))
display(expr, bac_cab(expr, False))

Again, should we wish to target only one occurence, we can pass the target as the second argument:

In [8]:
targets = find_bac_cab(expr)
display(targets)
bac_cab(expr, False, targets[1])

Here we used the function `find_bac_cab` to easily find all the occurences, then we selected the one we were interested in substituting.

## Dot-Cross rule

Another useful rule is:

$$
\vec{a} \cdot \left(\vec{b} \times \vec{c}\right) = \left(\vec{a} \times \vec{b}\right) \cdot \vec{c}
$$

To perform this substitution we can use the function `dot_cross`:

In [9]:
expr = (a & (b ^ c)) + (d & (e ^ f))
display(expr, dot_cross(expr))

In [10]:
expr = (a & (b ^ (c + d * (e & (g ^ h)))))
display(expr, dot_cross(expr))

## Expansion

Given a vector `expr`, we can use the method `expr.expand()` to expand its arguments:

In [11]:
expr = a ^ ((b + c) ^ d)
display(expr, expr.expand())

In [12]:
expr = (a + b) & (c + d)
display(expr, expr.expand())

In [13]:
expr = x * ((a & (b + (c & (d + e) * f.mag) * g)) + (((a + b) ^ c) & d))
display(expr, expr.expand())

## Collect

Because Sympy doesn't know what `VecDot` and `VecCross` are, the method `collect(expr, match)` has been implemented to overcome this problem.

If `match` is not an instance of `VecDot, VecCross, VecPow`, then it will fall back to Sympy `collect` method.

However, if `match` is an instance of `VecDot, VecCross, VecPow`, the result to be expected is very different wrt the standard collect. Let's see a few examples to understand what it produces. 

In [14]:
t = a.mag
expr = t * b + t * c + t**2 * d + t**2 * e + t**3 * f
display(expr, collect(expr, t))

In this first example, the `match` is not an instance of `VecDot, VecCross`, so the implementation falled-back to Sympy's collect implementation and it collected common powers of `match`.

The situation is very different if we try to collect a `VecDot` or `VecCross`:

In [15]:
expr = (a * (a & b)**3) + (b * (a & b)**2) + d
display(expr, collect(expr, a & b))

As we can see, the current implementation doesn't collect the common powers of `match`. Instead, it collects all the terms containing `match`. Another example:

In [16]:
expr = (a * (a & b)**3) + (b * (a & b)**2) + c * (a & b) + d
display(expr, collect(expr, (a & b)**2))

## Collect dot/cross products

Cross and dot products are distributive over addition. For example:

$$
\vec{a} \times \left(\vec{b} + \vec{c}\right) = \vec{a} \times \vec{b} + \vec{a} \times \vec{c}
$$

and:

$$
\vec{a} \cdot \left(\vec{b} + \vec{c}\right) = \vec{a} \cdot \vec{b} + \vec{a} \cdot \vec{c}
$$

As we have seen, we can move from the left hand side to the righ hand side with the method `expr.expand()`. To move from the right to the left we use the method `collect_cross_dot`, which accepts the following arguments:
* expr: the expression to modify;
* pattern: the class to target, it can be `VecCross` or `VecDot`. Default value to `VecCross`.
* first_arg : Boolean, default to True. If True, look for common first arguments in the instances of the class `pattern`. Otherwise, look for common second arguments.

Let's see a few examples:

In [17]:
expr = (a ^ b) + (a ^ c) + (b ^ c)
display(expr, collect_cross_dot(expr))

In [18]:
collect_cross_dot(expr, VecCross, False)

In [19]:
expr = (a ^ (b ^ c)) + (a ^ (b ^ d)) + (a ^ (b ^ e)) + (a ^ (e ^ d)) + (c ^ (d ^ e)) + (c ^ (d ^ f)) + (b ^ (c ^ d)) + (b ^ (c ^ e))
display(expr, collect_cross_dot(expr))

In [20]:
expr = (a & b) + (a & c) + (b & c)
collect_cross_dot(expr, VecDot)

In [21]:
collect_cross_dot(expr, VecDot, False)

## Simplify

In [22]:
simplify(a & a, dot_to_mag=False)

Finally, the method `simplify(expr, **kwargs)` can be used to apply different simplification rules simultaneously. Let's see what we can do with it:

In [23]:
help(simplify)

Help on function simplify in module vector_simplify:

simplify(expr, **kwargs)
    Apply a few simplification rules to expr.
    
    **kwargs:
        dot_to_mag : Default to True.
            a & a = a.mag**2
        bac_cab : Default to True.
            b * (a & c) - c * (a & b) = a ^ (b ^ c)
        coeff: Default to True
            (k1 * a) & (((k2 * b) & (k3 * c)) * d) = (k1 * k2 * k3) (a & ((b & c) * d))
            (k1 * a) ^ ((k2 * b) ^ (k3 * c)) = (k1 * k2 * k3) (a ^ (b ^ c))
        dot : Default to True
            (a & b) + (a & c) = a & (b + c)
            (a & b) + (c & b) = (a + c) & b
        cross : Default to True
            (a ^ b) + (a ^ c) = a ^ (b + c)
            (a ^ b) + (c ^ b) = (a + c) ^ b



In [24]:
expr = (a & a) + (b & b) + (a & b)
display(
    expr, 
    simplify(expr), 
    simplify(expr, dot_to_mag=False), 
    simplify(expr, dot_to_mag=False, dot=False)
)

In [25]:
expr = 4 * a ^ ((3 * (b + c)) ^ (2 * (3 * d ^ (x * e))))
display(
    expr, 
    simplify(expr), 
    simplify(expr, coeff=False)
)

In [26]:
expr = a.mag * ((a ^ (b ^ c)) + (a ^ (b ^ d))) + (d ^ e) + (f ^ e)
display(
    expr,
    simplify(expr),
    simplify(expr, cross=False)
)

In [27]:
expr = b * (a & c) - c * (a & b) + (e ^ (b ^ c))
display(
    expr,
    simplify(expr),
    simplify(expr, cross=False)
)

# Identities

To apply one of the following identities, use the method `identities(expr, **hints)` providing the appropriate hints:

$$
\begin{aligned}
& \nabla \cdot \left(\phi \vec{a}\right) = \phi \left(\nabla \cdot \vec{a}\right) + \left(\nabla \phi\right) \cdot \vec{a} \\
& \nabla \times \left(\phi \vec{a}\right) = \phi \left(\nabla \times \vec{a}\right) + \left(\nabla \phi\right) \times \vec{a} \\
& \nabla \cdot \left(\vec{a} \times \vec{b}\right) = \left(\nabla \times \vec{a}\right) \cdot \vec{b} - \left(\nabla \times \vec{b}\right) \cdot \vec{a} \\
& \nabla \times \left(\vec{a} \times \vec{b}\right) = - \left(\nabla \cdot \vec{a}\right) \vec{b} + \left(\nabla \cdot \vec{b}\right) \vec{a} - \left(\vec{a} \cdot \nabla\right)\vec{b} + \left(\vec{b} \cdot \nabla\right)\vec{a} \\
& \nabla \left(\vec{a} \cdot \vec{b}\right) = \left(\vec{a} \cdot \nabla\right)\vec{b} + \left(\vec{b} \cdot \nabla\right)\vec{a} + \vec{a} \times \left(\nabla \times \vec{b}\right) + \vec{b} \times \left(\nabla \times \vec{a}\right) \\
& \nabla \times \left(\nabla \times \vec{a}\right) = \nabla \left(\nabla \cdot \vec{a}\right) - \nabla^{2} \vec{a} \\
\end{aligned}
$$

In [28]:
help(identities)

Help on function identities in module vector_simplify:

identities(expr, **hints)
    Apply to expr the identities specified in **hints.
    
    In the following, w is scalar, wa/wb are vectors.
    
    Hints: set a flag to True to apply the identity. By default all
    flags are set to False.
    
        prod_div = True
            nabla & (w * wa) = (Grad(w) & wa) + (w * (nabla & wa))
        
        prod_curl = True
            nabla ^ (w * wa) = (Grad(w) ^ wa) + (w * (nabla ^ wa))
        
        div_of_cross = True
            nabla & (wa ^ wb) = ((nabla ^ wa) & wb) - ((nabla ^ wb) & wa)
        
        curl_of_cross = True
            nabla ^ (wa ^ wb) = ((nabla & wb) * wa) + DotNablaOp(wb, wa) - ((nabla & wa) * wb) - DotNablaOp(wa, wb)
        
        grad_of_dot = True
            Grad(wa & wb) = DotNablaOp(wa, wb) + DotNablaOp(wb, wa) + (wa ^ (nabla ^ wb)) + (wb ^ (nabla ^ wa))
        
        curl_of_grad = True
            nabla ^ Grad(w) = VectorZero()
    
        

For example:

In [29]:
nabla = Nabla()
expr = (nabla ^ (x * a)) + Grad(b & c) + (e ^ (f ^ g))
expr

In [30]:
identities(expr, prod_curl=True)

In [31]:
identities(expr, grad_of_dot=True)

In [32]:
identities(expr, prod_curl=True, grad_of_dot=True)